Análisis de equipos segunda división regional masculina
---

El objetivo es analizar los resultados e información sobre los equipos en temporadas anteriores, de cara a la panificación de la próxima temporada, conocer los equipos potencialmente más fuertes, los que han mejorado o empeorado en el tiempo, etc.

In [1]:
# Imports
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Cargar los DataFrames con los que vamos a trabajar
uri = 'sqlite:///../resources/data/fbrm.db'
results_df = pd.read_sql_table('results', uri)
teams_df = pd.read_sql_table('teams', uri)

### Mejores equipos 2022/2023

In [2]:
# Filtrar resultados para la temporada 2022/2023
season_2022_2023_df = results_df[results_df['season'] == '2022/2023']
# Columnas que nos interesan
colums = ['team', 'phase', 'group', 'position', 'win', 'scored', 'against']
season_2022_2023_df = season_2022_2023_df[colums]
# Equipos que han quedado primeros en cada una de la fases
season_2022_2023_df = season_2022_2023_df[season_2022_2023_df['position'] == 1]
season_2022_2023_df = season_2022_2023_df.sort_values(by=['phase', 'win'], ascending=False)
# Print
season_2022_2023_df

,team,phase,group,position,win,scored,against
76,CIKY ORO C.D. LOS URRUTIAS,REGULAR,B,1,12,938,697
114,MY ENGLISH HOUSE LA FLOTA CB EL CARMEN,REGULAR,A,1,12,888,615
106,MY ENGLISH HOUSE LA FLOTA CB EL CARMEN,2ª FASE,TITULO,1,12,1008,866
84,BAHIA MAZARRON BASKET,2ª FASE,CLASIFICACION,1,11,932,629


### Mejor equipo histórico

Para elegir el mejor equipo histórico se van a tener en cuenta, partidos ganados, posición y puntos anotados.

In [3]:
# Asignar pesos a cada criterio
weight_win = 0.5
weight_position = 0.3
weight_scored = 0.2

# Genera el nuevo DF para trabajar
historic_team_df = results_df.sort_values(by=['team','win','position','scored'],
                                            ascending=False)

# Agrupa los registros por equipo y calcula la media
historic_team_df = historic_team_df.groupby('team').agg({
    'win': 'mean',  # Promedio de partidos ganados
    'position': 'mean',         # Promedio de posición
    'scored': 'mean'   # Promedio de puntos anotados
}).reset_index()

# Eliminar los equipos que se han retirado de la competición, puesto que suman 0 puntos
historic_team_df = historic_team_df[~historic_team_df['team'].str.contains('retirado|RETIRADO')]

# Calcula la puntuación ponderada para cada equipo
historic_team_df['historic_score'] = (
    weight_win * historic_team_df['win'] +
    weight_position * (1 / historic_team_df['position']) +
    weight_scored * historic_team_df['scored']
)

# Ordenar por la puntuación obtenida
historic_team_df = historic_team_df.sort_values(by='historic_score', ascending=False)

# Print
historic_team_df.head()


,team,win,position,scored,historic_score
12,C.B. JAIRIS,22.0,1.0,2150.0,441.30
70,"UCAM MURCIA C.B. ""A"" (JUNIOR)",16.0,3.0,1978.0,403.70
6,AR ABOGADOS C.B.SANTOMERA,20.0,1.0,1914.0,393.10
51,LORCA F.C. AD ELIOCROCA,19.0,2.0,1858.0,381.25
69,UCAM MURCIA C.B.,14.0,5.0,1816.0,370.26


### Equipo más mejorado

Actualmente tenemos datos históricos desde la temporada 2015/2016 hasta la temporada 2022/2023. Por lo que se va a analizar que equipos han tenido mayor evolución durante esta franja de tiempo.

No tienen por qué coincidir con los mejores equipos.

Para evaluar cuanto ha mejorado un equipo, vamos a tener en cuenta victorias, posición, puntos anotados y puntos recibidos.

In [4]:
# Asignar pesos a cada criterio
weight_win = 0.3
weight_position = 0.4
weight_scored = 0.2
weight_against = 0.1

# Agrupa los registros por equipo y temporada, y calcula la media de los parámetros de evaluación
more_improved_rf = results_df.groupby(['season','team']).agg({
    'position': 'mean', # Promedio de posición
    'win': 'mean',      # Promedio de partidos ganados
    'scored': 'mean',   # Promedio de puntos anotados
    'against': 'mean'   # Promedio de puntos recibidos
}).reset_index()

# Normalizar los datos utilizando Min-Max Scaling
normalized_data = scaler.fit_transform(more_improved_rf[['position', 'win', 'scored', 'against']])
more_improved_rf[['position_norm', 'win_norm', 'scored_norm', 'against_norm']] = normalized_data

# Calcula la puntuación ponderada para cada equipo
more_improved_rf['season_score'] = (
    weight_position * (1 - more_improved_rf['position_norm']) +
    weight_win * more_improved_rf['win_norm'] +
    weight_scored * more_improved_rf['scored_norm'] +
    weight_against * (1 - more_improved_rf['against_norm'])
)

more_improved_rf = more_improved_rf.sort_values(by=['team', 'season'])

# Calcula la diferencia entre las puntuaciones ponderadas de temporadas sucesivas
more_improved_rf['improvement'] = more_improved_rf.groupby('team')['season_score'].diff()

# Encuentra el índice del equipo más mejorado
index_of_most_improved_team = more_improved_rf['improvement'].idxmax()

# Usa el índice para obtener el nombre del equipo más mejorado
most_improved_team_name = more_improved_rf.loc[index_of_most_improved_team, 'team']

# Print
print_colums = ['season', 'team', 'position', 'win', 'scored', 'against', 'season_score', 'improvement']
# more_improved_rf[print_colums]
more_improved_rf[more_improved_rf['team'] == most_improved_team_name][print_colums]

,season,team,position,win,scored,against,season_score,improvement
23,2017/2018,A.D. INFANTE,13.0,2.0,1441.0,1863.0,0.168494,NaN
36,2018/2019,A.D. INFANTE,3.0,18.0,1765.0,1645.0,0.761011,0.592517
62,2020/2021,A.D. INFANTE,3.0,6.0,686.5,620.0,0.548120,-0.212891


### Peor equipo histórico

In [5]:
# Print
historic_team_df.tail()

,team,win,position,scored,historic_score
9,BASKET LAS TORRES,1.0,7.5,647.5,130.040000
5,ALJEMA C.B. CARAVACA,4.0,5.0,633.5,128.760000
56,MESÓN DE MURCIA CB EL CARMEN,2.5,5.0,623.0,125.910000
59,PEUGEOT GRUPO MARCOS CBEC,1.0,6.5,623.0,125.146154
32,EBS CARTAGENA,3.5,3.5,516.0,105.035714


### Equipo más empeorado

In [6]:
# Encuentra el índice del equipo menos mejorado
index_of_less_improved_team = more_improved_rf['improvement'].idxmin()

# Usa el índice para obtener el nombre del equipo menos mejorado
less_improved_team_name = more_improved_rf.loc[index_of_most_improved_team, 'team']

# Print
print_colums = ['season', 'team', 'position', 'win', 'scored', 'against', 'season_score', 'improvement']
# more_improved_rf[print_colums]
more_improved_rf[more_improved_rf['team'] == less_improved_team_name][print_colums]

,season,team,position,win,scored,against,season_score,improvement
23,2017/2018,A.D. INFANTE,13.0,2.0,1441.0,1863.0,0.168494,NaN
36,2018/2019,A.D. INFANTE,3.0,18.0,1765.0,1645.0,0.761011,0.592517
62,2020/2021,A.D. INFANTE,3.0,6.0,686.5,620.0,0.548120,-0.212891


### Mejor equipo ofensivo

Mejor equipo ofensivo de cada temporada, para ello se va a tener en cuenta únicamente los puntos anotados, independientemente de si se traducen en victorias.

In [7]:

# Agrupa por temporada y equipo y realiza el sumatorio de los puntos anotados
best_offensive_df = results_df.groupby(['season', 'team'])['scored'].sum().reset_index()

# Agrupa los resultados por temporada y encuentra el índice del equipo con la máxima puntuación en cada temporada
idx_max_scored_team = best_offensive_df.groupby(['season'])['scored'].idxmax()

# Usa los índices para obtener los equipos correspondientes
teams_with_max_scored = best_offensive_df.loc[idx_max_scored_team, ['season', 'team', 'scored']]

# Print
teams_with_max_scored

,season,team,scored
10,2015/2016,TUTTOCARS MB,1789
16,2016/2017,HERO JAIRIS,1643
26,2017/2018,C.B. JAIRIS,2150
38,2018/2019,AR ABOGADOS C.B.SANTOMERA,1914
51,2019/2020,C.B.SANTOMERA,1604
66,2020/2021,ESP SOLUTIONS JAIRIS,1594
74,2021/2022,AD INFANTE PAN MORENO,1872
96,2022/2023,CLUB DEPORTIVO PUERTO DE MAZARRON,2070


### Mejor equipo defensivo

Mejor equipo defensivo de cada temporada, para ello se va a tener en cuenta el valor de puntos recibidos, independientemente de si se traduce en victorias.

In [8]:
# Agrupa por temporada y equipo y realiza el sumatorio de los puntos recibidos
best_defensive_df = results_df.groupby(['season', 'team'])['against'].sum().reset_index()

# Eliminar los equipos que se han retirado de la competición, puesto que suman 0 puntos
best_defensive_df = best_defensive_df[~best_defensive_df['team'].str.contains('retirado|RETIRADO')]

# Agrupa los resultados por temporada y encuentra el índice del equipo con la mínima puntuación en cada temporada
idx_min_against_team = best_defensive_df.groupby(['season'])['against'].idxmin()

# Usa los índices para obtener los equipos correspondientes
teams_with_min_against = best_defensive_df.loc[idx_min_against_team, ['season', 'team', 'against']]

# Print
teams_with_min_against

,season,team,against
7,2015/2016,HERO JAIRIS,1276
16,2016/2017,HERO JAIRIS,1107
26,2017/2018,C.B. JAIRIS,1404
38,2018/2019,AR ABOGADOS C.B.SANTOMERA,1637
56,2019/2020,F.P. AMUSAL CB SANTO ANGEL,1194
64,2020/2021,CRC MOLINA BASKET,1103
78,2021/2022,F.P. AMUSAL C.B. SANTO ANGEL,1355
98,2022/2023,"INMOBILIARIA ESTUDIO 21 ""CBT""",1290
